# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
mode = 'CIFAR10'

import os
import struct
import socket
import pickle
import time

import h5py
import numpy as np

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from torchvision.datasets import CIFAR10

## Define ECG dataset class

In [2]:
if mode == 'ECG':
    class ECG(Dataset):
        def __init__(self, train=True):
            if train:
                with h5py.File(os.path.join('../../', 'mitdb', 'train_ecg.hdf5'), 'r') as hdf:
                    self.x = hdf['x_train'][:]
                    self.y = hdf['y_train'][:]
            else:
                with h5py.File(os.path.join('../../', 'mitdb', 'test_ecg.hdf5'), 'r') as hdf:
                    self.x = hdf['x_test'][:]
                    self.y = hdf['y_test'][:]
        
        def __len__(self):
            return len(self.x)
        
        def __getitem__(self, idx):
            return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])
        
    batch_size = 32
    train_dataset = ECG(train=True)
    test_dataset = ECG(train=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    total_batch = len(train_loader)
    print(total_batch)
elif mode == 'CIFAR10':
    root_path = '../../data/'  # Replace with actual path to CIFAR10 data

    batch_size = 32

    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    train_dataset = CIFAR10(root=root_path, train=True, download=True, transform=transform)
    test_dataset = CIFAR10(root=root_path, train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print("Loaded CIFAR10 datasets with batch size", batch_size)

Files already downloaded and verified
Files already downloaded and verified
Loaded CIFAR10 datasets with batch size 32


In [3]:
if mode == 'ECG':
    class EcgClient(nn.Module):
        def __init__(self):
            super(EcgClient, self).__init__()
            self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
            self.relu1 = nn.ReLU()
            self.pool1 = nn.MaxPool1d(2)  # 64 x 16
            self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
            self.relu2 = nn.ReLU()
            self.pool2 = nn.MaxPool1d(2)  # 32 x 16
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.pool2(x)
            x = x.view(-1, 32 * 16)
            return x
elif mode == 'CIFAR10':
    class EcgClient(nn.Module):
        def __init__(self):
            super(EcgClient, self).__init__()
            self.conv1 = nn.Conv2d(3, 16, 7, padding=3)
            self.relu1 = nn.LeakyReLU()
            self.pool1 = nn.MaxPool2d(2)
            self.conv2 = nn.Conv2d(16, 16, 5, padding=2)
            self.relu2 = nn.LeakyReLU()
            self.pool2 = nn.MaxPool2d(2)
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.pool2(x)
            x = x.view(-1, 8 * 8 * 16)
            return x

In [4]:
device = torch.device('cuda:0')
torch.cuda.get_device_name(0)
assert('cuda' in device.type)
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
ecg_client = EcgClient()
ecg_client.to(device)

checkpoint = torch.load(f"../ecg_2conv/init_weight{mode}.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]
epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data
host = 'localhost'
port = 10080
max_recv = 4096
s = socket.socket()
s.connect((host, port))
print(len(train_loader))
print(len(test_loader))

1563
313


Training Process

In [5]:
from tqdm import tqdm
print(len(train_loader))
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in tqdm(enumerate(train_loader)):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        ecg_client = ecg_client.to(device)
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()
    print('done with batch')
            
    with torch.no_grad():  # calculate test accuracy
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

1563
Epoch 1 - 

1563it [00:09, 157.27it/s]


done with batch
loss: 2.1007, acc: 35.19% / test_loss: 0.4108, test_acc: 40.37%
Epoch 2 - 

1563it [00:09, 161.28it/s]


done with batch
loss: 2.0157, acc: 44.05% / test_loss: 0.3963, test_acc: 47.86%
Epoch 3 - 

1563it [00:09, 159.11it/s]


done with batch


In [ ]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
